In [2]:
import mxnet as mx
from mxnet import autograd,gluon,init,nd
from mxnet.gluon import loss as gloss ,nn
import time

ModuleNotFoundError: No module named 'mxnet'

In [2]:
net=nn.Sequential()
net.add(nn.Conv2D(kernel_size=5,activation='sigmoid',channels=6),
        nn.MaxPool2D(pool_size=2),
       nn.Conv2D(kernel_size=5,activation='sigmoid',channels=16),
       nn.MaxPool2D(pool_size=2),
       nn.Dense(120,activation='sigmoid'),
       nn.Dense(84,activation='sigmoid'),
       nn.Dense(10))

In [4]:
batch_size=256
train_iter,test_iter=gb.load_data_fashion_mnist(batch_size=batch_size)

In [6]:
def evaluate_accuracy(data_iter,net):
    acc=nd.array([0],ctx=mx.gpu())
    for X,y in data_iter:
        X=X.as_in_context(mx.gpu())
        y=y.as_in_context(mx.gpu())
        acc+=gb.accuracy(net(X),y)
    return acc.asscalar()

In [7]:
def train_ch5(net,train_iter,test_iter,batch_size,trainer,num_epochs):
    print('train on ' ,mx.gpu())
    loss=gloss.SoftmaxCrossEntropyLoss()
    for epoch in range(num_epochs):
        train_acc_sum,test_acc_sum=0,0
        start=time.time()
        for X,y in train_iter:
            X=X.as_in_context(mx.gpu())
            y=y.as_in_context(mx.gpu())
            with autograd.record():
                l=loss(net(X),y)
            l.backward()
            trainer.step(batch_size)
            train_acc_sum+=gb.accuracy(net(X),y)
        test_acc_sum=evaluate_accuracy(test_iter,net)
        print('epoch %d,train acc:%.3f ,test acc %.3f,time %.1f sec'%(epoch,train_acc_sum/len(train_iter),test_acc_sum/len(test_iter),time.time()-start))
            
    

In [8]:
lr,num_epochs=0.8,5
net.initialize(force_reinit=True,ctx=mx.gpu(),init=init.Xavier())
trainer=gluon.Trainer(net.collect_params(),'sgd',{'learning_rate':lr})
train_ch5(net,train_iter,test_iter,batch_size,trainer,num_epochs)


train on  gpu(0)
epoch 0,train acc:0.130 ,test acc 0.099,time 4.7 sec
epoch 1,train acc:0.305 ,test acc 0.571,time 2.7 sec
epoch 2,train acc:0.641 ,test acc 0.696,time 2.7 sec
epoch 3,train acc:0.719 ,test acc 0.725,time 2.7 sec
epoch 4,train acc:0.747 ,test acc 0.753,time 2.8 sec
